## Теоретическая часть

---

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


*Принципиальным отличием item-item и ALS методов является то, что item-item не является машинным обучением это алгоритм основанный на KNN, работающий в очень разреженной зашумленной матрице user-item, и поэтому работает очень медленно и ресурсозатратно, как результат он предсказывает вероятность числа из этой матрицы. В свою очередь, ALS это уже машинное обучение, и является способом оптимизации основанном на принципе минимизации среднеквадратичной ошибки на существующих рейтингах, как результат предсказывает не вероятности, а числа. Для больших данных ALS оказывается значительно быстрее в работе, чем item-item метод.*

---

2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


* Кол-во походов в магазин покупателя за товаром (basket_id, count)
* Суммарный объем продаж для покупателя товара (sales_value, sum)
* Отношение кол-ва походов в магазин покупателя за товаром (basket_id, count) к кол-ву купленных товаров (quantity, count)


---

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


* Предсказывает не вероятность, а некое число, которое определяет: чем больше число, тем более релевантен товар;
* Не учитывается сезонность;
* Не учитываются признаки покупателей или товаров.

---

## Практическая часть


---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys


In [2]:
data = pd.read_csv('../webinar_2/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(
    columns={
        'household_key': 'user_id',
        'product_id': 'item_id'
    },
    inplace=True
)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
print('\n'.join(data_train.columns.tolist()))

user_id
basket_id
day
item_id
quantity
sales_value
store_id
retail_disc
trans_time
week_no
coupon_disc
coupon_match_disc


In [4]:
popularity = data_train\
    .groupby('item_id', as_index=False)\
    ['quantity'].sum()\
    .rename(columns={'quantity': 'n_sold'})

k=5000

top_k = popularity\
    .sort_values('n_sold', ascending=False)\
    .head(k)\
    .item_id\
    .tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, 
# то он "купил" такой товар)

data_train.loc[
    ~data_train['item_id'].isin(top_k), 
    'item_id'
] = 999999

/home/sergey/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


In [5]:
result_train = data_train.groupby('user_id')['item_id'].unique().reset_index()
result_train.columns = ['user_id', 'actual']
result_train['n_item'] = result_train.actual.apply(len)
result_train = result_train[['user_id', 'n_item', 'actual']]
result_train.head(3)

,user_id,n_item,actual
0,1,235,"[999999, 840361, 845307, 852014, 856942, 91267..."
1,2,280,"[854852, 930118, 1077555, 1098066, 999999, 556..."
2,3,352,"[866211, 878996, 882830, 904360, 921345, 99999..."


In [6]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns = ['user_id', 'actual']
result_test['n_item'] = result_test.actual.apply(len)
result_test = result_test[['user_id', 'n_item', 'actual']]
result_test.head(3)

,user_id,n_item,actual
0,1,72,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,19,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,62,"[920308, 926804, 946489, 1006718, 1017061, 107..."


In [7]:
# Кол-во походов в магазин покупателя за товаром
user_item_matrix_1 = pd.pivot_table(
    data_train, 
    index='user_id', columns='item_id', 
    values='basket_id',
    aggfunc='count', 
    fill_value=0
)

# Суммарный объем продаж для покупателя товара
user_item_matrix_2 = pd.pivot_table(
    data_train, 
    index='user_id', columns='item_id', 
    values='sales_value',
    aggfunc='sum', 
    fill_value=0
)

# Отношение кол-ва походов в магазин покупателя за товаром 
# к кол-ву купленных товаров
pvt = pd.pivot_table(
    data_train, 
    index='user_id', columns='item_id', 
    values=['basket_id', 'quantity'],
    aggfunc={
        'basket_id': 'count',
        'quantity': 'count'
    },
    fill_value=0
)

user_item_matrix_3 = pvt.basket_id / pvt.quantity
user_item_matrix_3.fillna(0, inplace=True)

In [8]:
def get_recommendations(user, model, N=5):
    result = [id_to_itemid[rec[0]] for rec in
              model.recommend(
                  userid=userid_to_id[user],
                  user_items=sparse_user_item,
                  N=N,
                  filter_already_liked_items=False,
                  filter_items=[itemid_to_id[999999]],
                  recalculate_user=True
              )]
    return result

In [9]:
user_item_matrixs = [
    user_item_matrix_1, 
    user_item_matrix_2, 
    user_item_matrix_3
]

for i, user_item_matrix in enumerate(user_item_matrixs, 1):
    
    user_item_matrix = user_item_matrix.astype(float)
    
    sparse_user_item = csr_matrix(user_item_matrix).tocsr()
    
    userids = user_item_matrix.index.values
    itemids = user_item_matrix.columns.values

    matrix_userids = np.arange(len(userids))
    matrix_itemids = np.arange(len(itemids))

    id_to_itemid = dict(zip(matrix_itemids, itemids))
    id_to_userid = dict(zip(matrix_userids, userids))

    itemid_to_id = dict(zip(itemids, matrix_itemids))
    userid_to_id = dict(zip(userids, matrix_userids))

    model = AlternatingLeastSquares(
        factors=44, 
        regularization=0.001,
        iterations=15, 
        calculate_training_loss=True, 
        use_gpu=False)

    model.fit(
        csr_matrix(user_item_matrix).T.tocsr(),
        show_progress=True
    )    
    
    result_train[f'als_{i}'] = result_train['user_id']\
        .apply(lambda x: get_recommendations(x, model=model, N=5))
    
    result_test[f'als_{i}'] = result_test['user_id']\
        .apply(lambda x: get_recommendations(x, model=model, N=5))

In [10]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [11]:
def precision_model(result):
    dict_precision_at_k = {
        col: round(result.apply(
        lambda row: precision_at_k(row[col], row['actual']), 
        axis=1).mean(), 4) for col in result.columns[3:].tolist()
    }

    df = pd.DataFrame(
        pd.Series(dict_precision_at_k), 
        columns=['precision_at_k']
    )

    print(df)
    print()
    print(f'Вывод:\nЛучшая модель {df.precision_at_k.sort_values(ascending=False).index[0]}')

In [12]:
precision_model(result_train)

       precision_at_k
als_1          0.7046
als_2          0.5684
als_3          0.8463

Вывод:
Лучшая модель als_3


In [13]:
precision_model(result_test)

       precision_at_k
als_1          0.1838
als_2          0.1168
als_3          0.2141

Вывод:
Лучшая модель als_3


Вывод:
* Лучшую метрику показала модель №3.

Веса в user-item матрице заданы как отношение кол-ва походов в магазин покупателя за товаром к кол-ву купленных товаров

### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [14]:
user_item_matrix = pd.pivot_table(
    data_train, 
    index='user_id', columns='item_id', 
    values=['basket_id', 'quantity'],
    aggfunc={
        'basket_id': 'count',
        'quantity': 'count'
    },
    fill_value=0
)

user_item_matrix = user_item_matrix.basket_id / user_item_matrix.quantity
user_item_matrix.fillna(0, inplace=True)

user_item_matrix = user_item_matrix.astype(float)
    
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params_model = {
    'factors': [50, 100, 300],
    'regularization': [0.001, 0.0015, 0.002],
    'iterations': [15, 30, 45],
    'calculate_training_loss': [True],
    'use_gpu': [False]
}

rand_search_model = RandomizedSearchCV(
    AlternatingLeastSquares(),
    param_distributions=params_model,
    n_iter=15,
    cv=6,
    refit=False
)

search_model = rand_search_model.fit(
    csr_matrix(user_item_matrix).T.tocsr(),
    show_progress=True
)

params = search_model.best_params_
params

# Пишет, что у модели ALS нет score_a ???

In [15]:
max_precision = 0

factors_list = [50, 100, 300]
regularization_list = [0.001, 0.002, 0.006]
iterations_list = [15, 30, 90]

sp = []

k = len(factors_list) * len(regularization_list) * len(iterations_list)

for factors in factors_list:
    for regularization in regularization_list:
        for iterations in iterations_list:
            
            model = AlternatingLeastSquares(
                factors=factors, 
                regularization=regularization,
                iterations=iterations, 
                calculate_training_loss=True, 
                use_gpu=False
            )
            
            model.fit(
                csr_matrix(user_item_matrix).T.tocsr(),
                show_progress=False
            )
            
            result_test['als'] = result_test['user_id'].apply(
                lambda x: get_recommendations(x, model=model, N=5)
            )
            
            precision = result_test.apply(
                lambda row: precision_at_k(
                    row['als'], 
                    row['actual']
                ), 
                axis=1
            ).mean()
            
            
            sp.append([round(precision, 4), model.factors, 
                       model.regularization, model.iterations])
            
            if precision > max_precision:
                
                max_precision = precision
                
                best_factors = factors
                best_regularization = regularization
                best_iterations = iterations
            
            print(k, end=', ')
            k-=1
            
best_params = {
    'factors': best_factors,
    'regularization': best_regularization, 
    'iterations': best_iterations
}

27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 

In [16]:
print('  Максимальное значение метрики precision_at_k достигается при следующих параметрах:\n')
for i, j in best_params.items(): print(f'{i}:\t{j:>9}')
print(f'\n  И имееет значение:\n\nmax_precision_@k = {round(max_precision, 4)}')

  Максимальное значение метрики precision_at_k достигается при следующих параметрах:

factors:	       50
regularization:	    0.006
iterations:	       15

  И имееет значение:

max_precision_@k = 0.2175


**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!

In [17]:
df = pd.DataFrame(sp,columns=['precision', 'factors', 
                                'regularization', 'iterations'])
df

,precision,factors,regularization,iterations
0,0.2130,50,0.001,15
1,0.2085,50,0.001,30
2,0.2151,50,0.001,90
3,0.2148,50,0.002,15
4,0.2119,50,0.002,30
5,0.2122,50,0.002,90
6,0.2175,50,0.006,15
7,0.2087,50,0.006,30
8,0.2136,50,0.006,90
9,0.2058,100,0.001,15


In [18]:
pd.DataFrame(df.groupby('factors')['precision'].mean())

,precision
factors,
50,0.212811
100,0.203267
300,0.165767


In [19]:
pd.DataFrame(df.groupby('regularization')['precision'].mean())

,precision
regularization,
0.001,0.194478
0.002,0.193233
0.006,0.194133


In [20]:
pd.DataFrame(df.groupby('iterations')['precision'].mean())

,precision
iterations,
15,0.195033
30,0.192822
90,0.193989


Вывод:
* Увеличение числа факторов приводит к уменьшению метрки. Поэтому в дальнейшем стоит провести исследование по уменьшению данного параметра
* Увеличение других параметров большого влияния на метрику не оказало.

In [21]:
!pip install hyperopt

     |████████████████████████████████| 1.6 MB 1.0 MB/s eta 0:00:01     |█████▉                          | 286 kB 1.0 MB/s eta 0:00:02     |███████████████                 | 747 kB 1.0 MB/s eta 0:00:01     |██████████████████████▊         | 1.1 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 27.9 MB/s eta 0:00:01


# Use hyperopt!